In [1]:
import heapq
from itertools import permutations

graph_search = {
    1: {2: 20, 3: 15, 4: 10},
    2: {1: 20, 3: 35, 4: 25},
    3: {1: 15, 2: 35, 4: 30},
    4: {1: 10, 2: 25, 3: 30}
}

def dfs(graph, start, goal, visited=None):
    if visited is None:
        visited = set()
    visited.add(start)
    if start == goal:
        return [start]
    for neighbor in graph[start]:
        if neighbor not in visited:
            path = dfs(graph, neighbor, goal, visited)
            if path:
                return [start] + path
    return None

def dls(graph, start, goal, depth, visited=None):
    if visited is None:
        visited = set()
    if depth < 0:
        return None
    visited.add(start)
    if start == goal:
        return [start]
    for neighbor in graph[start]:
        if neighbor not in visited:
            path = dls(graph, neighbor, goal, depth - 1, visited)
            if path:
                return [start] + path
    return None

def ucs(graph, start, goal):
    priority_queue = [(0, start, [])]  # (cost, node, path)
    visited = set()
    while priority_queue:
        cost, node, path = heapq.heappop(priority_queue)
        if node in visited:
            continue
        path = path + [node]
        if node == goal:
            return path, cost
        visited.add(node)
        for neighbor, weight in graph[node].items():
            if neighbor not in visited:
                heapq.heappush(priority_queue, (cost + weight, neighbor, path))
    return None

print("DFS Path:", dfs(graph_search, 1, 3))
print("DLS Path:", dls(graph_search, 1, 3, 2))
print("UCS Path:", ucs(graph_search, 1, 3))

DFS Path: [1, 2, 3]
DLS Path: [1, 2, 3]
UCS Path: ([1, 3], 15)


In [2]:
graph_tsp = {
    1: {2: 20, 3: 15, 4: 10},
    2: {1: 20, 3: 35, 4: 25},
    3: {1: 15, 2: 35, 4: 30},
    4: {1: 10, 2: 25, 3: 30}
}

def tsp(graph, start):
    nodes = list(graph.keys())
    nodes.remove(start)
    min_cost = float('inf')
    best_path = []
    for perm in permutations(nodes):
        cost = 0
        prev = start
        for node in perm:
            cost += graph[prev][node]
            prev = node
        cost += graph[prev][start]  # Return to start
        if cost < min_cost:
            min_cost = cost
            best_path = [start] + list(perm) + [start]
    return best_path, min_cost

print("TSP Shortest Path:", tsp(graph_tsp, 1))

TSP Shortest Path: ([1, 3, 2, 4, 1], 85)


In [3]:
def iddfs(graph, start, goal, max_depth):
    for depth in range(max_depth + 1):
        visited = set()
        path = dls(graph, start, goal, depth, visited)
        if path:
            return path
    return None

def bidirectional_search(graph, start, goal):
    if start == goal:
        return [start]
    frontier_f = {start}
    frontier_b = {goal}
    parents_f = {start: None}
    parents_b = {goal: None}
    while frontier_f and frontier_b:
        if frontier_f & frontier_b:
            intersection = (frontier_f & frontier_b).pop()
            path_f = []
            path_b = []
            node = intersection
            while node:
                path_f.append(node)
                node = parents_f[node]
            node = parents_b[intersection]
            while node:
                path_b.append(node)
                node = parents_b[node]
            return path_f[::-1] + path_b
        new_frontier_f = set()
        for node in frontier_f:
            for neighbor in graph[node]:
                if neighbor not in parents_f:
                    parents_f[neighbor] = node
                    new_frontier_f.add(neighbor)
        frontier_f = new_frontier_f
        new_frontier_b = set()
        for node in frontier_b:
            for neighbor in graph[node]:
                if neighbor not in parents_b:
                    parents_b[neighbor] = node
                    new_frontier_b.add(neighbor)
        frontier_b = new_frontier_b
    return None

print("IDDFS Path:", iddfs(graph_search, 1, 3, 3))
print("Bidirectional Search Path:", bidirectional_search(graph_search, 1, 3))


IDDFS Path: [1, 3]
Bidirectional Search Path: [1, 2, 3]
